Tercer ejercicio de Spark Structured Streaming. WordCount por ventana de tiempo con marca de agua (watermark)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import window

spark = SparkSession.builder.appName("StructuredNetworkWordCountWindowed").getOrCreate()

In [ ]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
# In command line -> nc -lk 9999
lines =spark.readStream.format("socket").option("host", "localhost").option("port", 9999).option('includeTimestamp', 'true').load()

In [ ]:
# Split the lines into words
words =lines.select(explode(split(lines.value, " ")).alias("word"), lines.timestamp)

In [ ]:
# Group the data by window and word and compute the count of each group
# Se utiliza la columna timestamp  añadida mediante la opción -> option('includeTimestamp', 'true')
# como marca de agua para a partir de esta campo hacer el filtrado 
# de todos aquellos registros que lleguen 10 minutos tarde con respecto a la ventana de tiempo
windowedCounts = words \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window(words.timestamp, "10 minutes", "5 minutes"),
        words.word) \
    .count()

In [ ]:
# Start running the query that prints the running counts to the console
# Ver resultados en la consola donde se ejecuto el jupyter notebook
query = windowedCounts.writeStream.outputMode("complete").format("console").option('truncate', 'false').start()
query.awaitTermination()